In [24]:
import sqlite3
import pandas as pd
import unicodedata
import ollama
import os
from rapidfuzz import process
from vanna.chromadb import ChromaDB_VectorStore
from vanna.ollama import Ollama

# ==========================================
# AGENTE 1: ANALISTA SQL (Expert em Dados)
# ==========================================
class SQLAnalyst(ChromaDB_VectorStore, Ollama):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        Ollama.__init__(self, config=config)

    def preparar_agente(self, db_path):
        self.connect_to_sqlite(db_path)
        
        # Extração de Metadados Reais para Grounding
        df_meta = self.run_sql("SELECT DISTINCT cidade, bairro, rua, especificacao FROM core_imovel")
        self.cidades = df_meta['cidade'].dropna().unique().tolist()
        self.bairros = df_meta['bairro'].dropna().unique().tolist()
        self.ruas = df_meta['rua'].dropna().unique().tolist()
        self.tipos = df_meta['especificacao'].dropna().unique().tolist()
        self.entidades = self.cidades + self.bairros + self.ruas

        if self.get_training_data().empty:
            print("--- Treinando o modelo com DDL e Regras de Negócio ---")
            self.train(ddl="""
            CREATE TABLE core_imovel (
                id INTEGER PRIMARY KEY AUTOINCREMENT, 
                titulo VARCHAR(200), 
                descricao TEXT,
                quartos INTEGER, 
                banheiros INTEGER, 
                garagem INTEGER, 
                area DECIMAL, 
                bairro VARCHAR(100), 
                rua VARCHAR(100), 
                preco_aluguel DECIMAL, 
                preco_iptu DECIMAL, 
                preco_condominio DECIMAL, 
                aceita_pets BOOLEAN,
                especificacao VARCHAR(100)
            );
            """)

            self.train(documentation=f"""
            - Localização: Juiz de Fora, MG.
            - Regra de Negócio: 'Barato' significa preco_aluguel < (SELECT AVG(preco_aluguel) FROM core_imovel).
            - IMPORTANTE: NÃO adicione filtros de 'aceita_pets' (0 ou 1) a menos que o usuário use as palavras 'pet', 'animal', 'cachorro' ou 'gato'.
            - Use sempre LIKE '%termo%' para textos.
            - Bairros válidos: {", ".join(self.bairros)}
            - Tipos válidos: {", ".join(self.tipos)}
            """)

    def normalizar(self, texto):
        nfkd = unicodedata.normalize('NFKD', str(texto))
        return "".join([c for c in nfkd if not unicodedata.combining(c)]).lower().strip()

    def fuzzy_cleanup(self, pergunta):
        """Corrige termos geográficos sem duplicar palavras."""
        tokens = pergunta.split()
        resultado = []
        protegidos = {"quarto", "banheiro", "vaga", "garagem", "aluguel", "pets", "casa", "apto", "rua", "bairro"}
        
        for t in tokens:
            t_clean = self.normalizar(t)
            if t_clean.isdigit() or t_clean in protegidos or len(t_clean) <= 3:
                resultado.append(t)
            else:
                match = process.extractOne(t_clean, [self.normalizar(e) for e in self.entidades], score_cutoff=88)
                if match:
                    # Recupera o nome original do banco de dados
                    idx = [self.normalizar(e) for e in self.entidades].index(match[0])
                    entidade_real = self.entidades[idx]
                    # Evita duplicar se a palavra já estiver no resultado (ex: Rua Rua)
                    if entidade_real not in resultado:
                        resultado.append(entidade_real)
                else:
                    resultado.append(t)
        return " ".join(resultado)

    def executar_consulta(self, pergunta):
        pergunta_limpa = self.fuzzy_cleanup(pergunta)
        try:
            sql = self.generate_sql(pergunta_limpa)
            df = self.run_sql(sql)
            return df, sql
        except Exception as e:
            return None, f"Erro: {str(e)}"

# ==========================================
# AGENTE 2: BIA (Secretária Conversacional)
# ==========================================
class BiaPersona:
    def __init__(self, bairros_validos, model_name='deepseek-r1:8b'):
        self.model = model_name
        self.bairros_validos = bairros_validos
        # System Prompt com Grounding Geográfico Rígido
        self.system_prompt = f"""
        Você é a Bia, secretária virtual da imobiliária em Juiz de Fora.
        REGRAS RÍGIDAS DE CONVERSA:
        1. NUNCA invente bairros. Se não houver imóveis, sugira APENAS bairros desta lista: {", ".join(self.bairros_validos)}.
        2. Se os dados parecerem absurdos (ex: área de 1m² ou kitnet com 4 quartos), descreva o imóvel de forma simples sem enfatizar esses números.
        3. Nunca cite SQL ou nomes técnicos de colunas (use 'aluguel' em vez de 'preco_aluguel').
        4. Se o resultado for uma soma (ex: total de aluguel), apenas informe o valor de forma gentil.
        5. Se o resultado for vazio, peça desculpas e ofereça bairros da lista acima.
        """

    def responder(self, pergunta, df):
        contexto = df.to_dict(orient='records') if df is not None and not df.empty else "Nenhum imóvel encontrado."
        prompt = f"Pergunta do Cliente: {pergunta}\nDados Reais do Banco: {contexto}\nResponda como a Bia:"
        
        try:
            # Temperatura baixa (0.2) para evitar alucinações criativas
            response = ollama.generate(
                model=self.model, 
                system=self.system_prompt, 
                prompt=prompt,
                options={'temperature': 0.2} 
            )
            return response['response'].split("</thought>")[-1].strip()
        except Exception:
            return "Ops, tive um pequeno problema técnico, mas já estou voltando! Como posso te ajudar?"

# ==========================================
# EXECUÇÃO E RELATÓRIO
# ==========================================
if __name__ == "__main__":
    # Configuração do Analista (Temp 0.0 para precisão no SQL)
    config_analista = {"model": "qwen2.5-coder:7b", "path": "./vanna_chroma_final", "temperature": 0.0}
    analista = SQLAnalyst(config=config_analista)
    analista.preparar_agente("db.sqlite3")
    
    # Configuração da Bia (Passando bairros reais para evitar alucinação)
    bia = BiaPersona(bairros_validos=analista.bairros)

    # Exemplo de fluxo de teste
    testes = [
        "Quero um apartamento no bairro São Mateus que aceite pets",
        "Qual o valor total de aluguel no Granbery?",
        "Tem kitnet barata no Centro?",
        "Imóveis na Rua Martins Barbosa"
    ]

    print("\n--- INICIANDO TESTES AJUSTADOS ---\n")
    for p in testes:
        print(f"Cliente: {p}")
        df, sql = analista.executar_consulta(p)
        print(f"SQL: {sql}")
        resposta = bia.responder(p, df)
        print(f"Bia: {resposta}\n" + "-"*30)

    # Nota: Para rodar a bateria de 150, basta usar a função 'gerar_bateria_testes' do código anterior.


--- INICIANDO TESTES AJUSTADOS ---

Cliente: Quero um apartamento no bairro São Mateus que aceite pets
[{'role': 'system', 'content': "The user provides a question and you provide SQL. You will only respond with SQL code and not with any explanations.\n\nRespond with only SQL code. Do not answer with any explanations -- just the code.\n\nYou may use the following DDL statements as a reference for what tables might be available. Use responses to past questions also to guide you:\n\n\n            CREATE TABLE core_imovel (\n                id INTEGER PRIMARY KEY AUTOINCREMENT, \n                titulo VARCHAR(200), \n                descricao TEXT,\n                quartos INTEGER, \n                banheiros INTEGER, \n                garagem INTEGER, \n                area DECIMAL, \n                bairro VARCHAR(100), \n                rua VARCHAR(100), \n                preco_aluguel DECIMAL, \n                preco_iptu DECIMAL, \n                preco_condominio DECIMAL, \n      

In [29]:
import sqlite3
import pandas as pd
import unicodedata
import ollama
import os
from rapidfuzz import process
from vanna.chromadb import ChromaDB_VectorStore
from vanna.ollama import Ollama

# ==========================================
# AGENTE 1: ANALISTA SQL (Versão Final 3.0)
# ==========================================
class SQLAnalyst(ChromaDB_VectorStore, Ollama):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        Ollama.__init__(self, config=config)

    def preparar_agente(self, db_path):
        """Conecta e treina com regras de negócio blindadas contra erros de tipagem."""
        self.connect_to_sqlite(db_path)
        
        # Extração de metadados reais para o Fuzzy Match
        df_meta = self.run_sql("SELECT DISTINCT bairro, rua, especificacao FROM core_imovel")
        self.bairros = [str(x) for x in df_meta['bairro'].dropna().unique().tolist()]
        self.ruas = [str(x) for x in df_meta['rua'].dropna().unique().tolist()]
        self.tipos = [str(x) for x in df_meta['especificacao'].dropna().unique().tolist()]
        self.entidades = self.bairros + self.ruas + self.tipos

        if self.get_training_data().empty:
            # Treinamento de DDL (Baseado na estrutura real do db.sqlite3)
            self.train(ddl="""
            CREATE TABLE core_imovel (
                id INTEGER PRIMARY KEY AUTOINCREMENT, 
                titulo VARCHAR(200), 
                descricao TEXT,
                quartos INTEGER, 
                banheiros INTEGER, 
                garagem INTEGER, 
                area DECIMAL, 
                bairro VARCHAR(100), 
                rua VARCHAR(100), 
                preco_aluguel DECIMAL, 
                preco_iptu DECIMAL, 
                preco_condominio DECIMAL, 
                aceita_pets BOOLEAN, -- 1 para Sim, 0 para Não
                especificacao VARCHAR(100) -- apartamento, casa, kitnet, studio, loft, cobertura
            );
            """)

            # Treinamento de Regras Críticas (Resolvendo falhas de auditoria)
            self.train(documentation=f"""
            - Localização: Juiz de Fora, MG.
            - REGRA DE ID: O campo 'id' é um INTEIRO. Ex: 'imóvel 131' deve ser traduzido como WHERE id = 131.
            - REGRA DE PETS: Se o cliente citar 'gato', 'cachorro' ou 'pets', use 'aceita_pets = 1'. 
            - NUNCA use LOWER() ou LIKE em colunas booleanas (aceita_pets) ou numéricas (preços, quartos, id).
            - Use LOWER() apenas para colunas de texto: bairro, rua, especificacao.
            - Custo Total = (preco_aluguel + preco_condominio + preco_iptu).
            - NUNCA adicione filtros de pet (aceita_pets = 0) a menos que o cliente peça 'que NÃO aceitem pets'.
            - Bairros em JF: {", ".join(self.bairros)}.
            """)

    def normalizar(self, texto):
        nfkd = unicodedata.normalize('NFKD', str(texto))
        return "".join([c for c in nfkd if not unicodedata.combining(c)]).lower().strip()

    def fuzzy_cleanup(self, pergunta):
        """Corrige a pergunta sem duplicar entidades ou alucinar bairros."""
        tokens = pergunta.split()
        resultado = []
        
        # Mapeamento rápido de tokens protegidos e numéricos
        for t in tokens:
            t_norm = self.normalizar(t)
            if t_norm.isdigit() or len(t_norm) <= 3:
                resultado.append(t)
                continue
            
            # Busca correspondência em bairros/ruas/tipos
            match = process.extractOne(t_norm, [self.normalizar(e) for e in self.entidades], score_cutoff=90)
            if match:
                # Recupera o nome original com a capitalização correta do banco
                idx = [self.normalizar(e) for e in self.entidades].index(match[0])
                entidade_real = self.entidades[idx]
                resultado.append(entidade_real)
            else:
                resultado.append(t)
        
        pergunta_limpa = " ".join(resultado)
        # Injeção semântica para Pets se houver menção a animais
        if any(x in pergunta.lower() for x in ["gato", "cachorro", "animal"]):
            pergunta_limpa += " que aceita pets"
            
        return pergunta_limpa

    def executar_consulta(self, pergunta):
        pergunta_limpa = self.fuzzy_cleanup(pergunta)
        try:
            sql = self.generate_sql(pergunta_limpa)
            df = self.run_sql(sql)
            return df, sql
        except Exception as e:
            return None, f"Erro: {str(e)}"

# ==========================================
# AGENTE 2: BIA (Persona Geofenced)
# ==========================================
class BiaPersona:
    def __init__(self, bairros_validos, model_name='deepseek-r1:8b'):
        self.model = model_name
        self.bairros_validos = bairros_validos
        self.system_prompt = f"""
        Você é a Bia, secretária virtual de uma imobiliária em Juiz de Fora.
        REGRAS:
        1. Se o banco de dados retornar 'Vazio', não invente dados. Diga que não encontrou e sugira bairros como: {", ".join(self.bairros_validos[:5])}.
        2. Nunca use termos técnicos de programação.
        3. Para cálculos, use os valores de aluguel, IPTU e condomínio fornecidos.
        """

    def responder(self, pergunta, df):
        contexto = df.to_dict(orient='records') if df is not None and not df.empty else "Nenhum imóvel encontrado."
        prompt = f"Pergunta do Cliente: {pergunta}\nDados Reais do Banco: {contexto}\nBia, responda:"
        
        try:
            response = ollama.generate(model=self.model, system=self.system_prompt, prompt=prompt, options={'temperature': 0.1})
            return response['response'].split("</thought>")[-1].strip()
        except Exception:
            return "Tive uma falha técnica rápida, mas posso pesquisar outro bairro para você em JF!"

# ==========================================
# MOTOR DE TESTES DE CONFERÊNCIA
# ==========================================
def bateria_de_conferencia(analista, bia):
    testes = [
        "Qual o custo total do imóvel 131?",              # Foco: Cálculo e ID Inteiro
        "Tem cobertura no Benfica que aceita gatos?",      # Foco: Regra de Pet Booleana
        "Quais casas tem no bairo Benfika?",              # Foco: Fuzzy Match sem alucinação
        "Qual o apartamento mais barato no Centro?"       # Foco: Ordenação e Filtro Geográfico
    ]
    
    print("\n📝 Iniciando Testes de Conferência Final...")
    for i, p in enumerate(testes, 1):
        df, sql = analista.executar_consulta(p)
        resposta = bia.responder(p, df)
        print(f"\n--- Teste {i} ---")
        print(f"Pergunta: {p}")
        print(f"SQL: {sql}")
        print(f"Bia: {resposta}")

if __name__ == "__main__":
    config_sql = {"model": "qwen2.5-coder:7b", "path": "./vanna_chroma_final_v3", "temperature": 0.0}
    analista = SQLAnalyst(config=config_sql)
    analista.preparar_agente("db.sqlite3")
    
    bia = BiaPersona(bairros_validos=analista.bairros)
    bateria_de_conferencia(analista, bia)

Adding ddl: 
            CREATE TABLE core_imovel (
                id INTEGER PRIMARY KEY AUTOINCREMENT, 
                titulo VARCHAR(200), 
                descricao TEXT,
                quartos INTEGER, 
                banheiros INTEGER, 
                garagem INTEGER, 
                area DECIMAL, 
                bairro VARCHAR(100), 
                rua VARCHAR(100), 
                preco_aluguel DECIMAL, 
                preco_iptu DECIMAL, 
                preco_condominio DECIMAL, 
                aceita_pets BOOLEAN, -- 1 para Sim, 0 para Não
                especificacao VARCHAR(100) -- apartamento, casa, kitnet, studio, loft, cobertura
            );
            
Adding documentation....

📝 Iniciando Testes de Conferência Final...
[{'role': 'system', 'content': "The user provides a question and you provide SQL. You will only respond with SQL code and not with any explanations.\n\nRespond with only SQL code. Do not answer with any explanations -- just the code.\n\nY

In [23]:
import sqlite3
import pandas as pd

# Conecta ao banco de dados
conn = sqlite3.connect('db.sqlite3')

# Carrega os dados da tabela core_imovel para um DataFrame
df = pd.read_sql_query("SELECT * FROM core_imovel", conn)

# Fecha a conexão
conn.close()

# Exibe as primeiras linhas
df.head()

,id,titulo,descricao,quartos,banheiros,garagem,area,cidade,bairro,rua,numero,preco_aluguel,preco_iptu,preco_condominio,aceita_pets,imagem,codigo_bairro,especificacao
0,59,Apartamento Confortável em São Mateus,"Ótimo apartamento de frente, sol da manhã, pró...",2,2,1,85.0,Juiz de Fora,São Mateus,Rua Padre Café,120,1800.0,150.0,350.0,1,None,200,apartamento
1,60,Cobertura Duplex São Mateus,"Cobertura incrível com vista panorâmica, área ...",3,3,2,160.0,Juiz de Fora,São Mateus,Rua Dr. Romualdo,450,3200.0,300.0,600.0,1,None,200,apartamento
2,62,Casa Charmosa no Alto dos Passos,"Casa antiga reformada, com quintal arborizado ...",3,2,2,200.0,Juiz de Fora,Alto dos Passos,Rua Severiano Sarmento,88,2500.0,220.0,0.0,1,None,220,casa
3,63,Apartamento Luxo Alto dos Passos,"Apartamento de alto padrão, um por andar, acab...",4,4,3,220.0,Juiz de Fora,Alto dos Passos,Rua Dom Viçoso,300,4500.0,500.0,1200.0,1,None,220,apartamento
4,64,Loft Industrial,"Loft com pegada industrial, pé direito duplo e...",1,1,1,70.0,Juiz de Fora,Alto dos Passos,Rua Morais e Castro,55,1900.0,120.0,400.0,0,None,220,apartamento
